### Goal: Summaraize text and predict what category it is out of business, entertainemnt, politics, sport and tech

In [18]:
import os
import pandas as pd
from pathlib import Path

In [19]:
data_path = Path(r"C:\Users\bamilosin\Documents\dataset\nlp\summarization\BBC News Summary")
articles_path = data_path / "News Articles"
summaries_path = data_path / "Summaries"

In [24]:
test_df = pd.DataFrame({
    "text": ["a ridiculously long text", "another ridiculously long text"],
    "summary" : ["long text", "another long text"],
    "category" : ["normal", "normal"]
})
test_df

,text,summary,category
0,a ridiculously long text,long text,normal
1,another ridiculously long text,another long text,normal


In [25]:
# open each dir /category
# open each file xyz.txt
# append file to data_dictionary 
# {
#     "text": ["a ridiculously long text", "another ridiculously long text"],
#     "summary" : ["long text", "another long text"],
#     "category" : ["normal", "normal"]
# }

In [ ]:
data_dict = {
    'text': [],
    'summary': [],
    'category': []
}

text_and_summary = [summaries_path, articles_path]
categories = ["business", "entertainment", "politics", "sport", "tech"]


for dir in text_and_summary:
    for category in categories:
        for file in os.listdir(dir / category):
            with open (dir / category / file, 'r') as f:
                file = f.readlines()
                data_dict['text']

In [51]:
file

['Ink helps drive democracy in Asia\n',
 '\n',
 "The Kyrgyz Republic, a small, mountainous state of the former Soviet republic, is using invisible ink and ultraviolet readers in the country's elections as part of a drive to prevent multiple voting.\n",
 '\n',
 'This new technology is causing both worries and guarded optimism among different sectors of the population. In an effort to live up to its reputation in the 1990s as "an island of democracy", the Kyrgyz President, Askar Akaev, pushed through the law requiring the use of ink during the upcoming Parliamentary and Presidential elections. The US government agreed to fund all expenses associated with this decision.\n',
 '\n',
 'The Kyrgyz Republic is seen by many experts as backsliding from the high point it reached in the mid-1990s with a hastily pushed through referendum in 2003, reducing the legislative branch to one chamber with 75 deputies. The use of ink is only one part of a general effort to show commitment towards more open 